In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
from tqdm.auto import tqdm
import numpy as np
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
from skimage.color import deltaE_cie76

In [ ]:
class LabDataset(Dataset):
    def __init__(self, npy_file):
        self.data = np.load(npy_file, allow_pickle=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        mp = self.data[idx, :3]  # First three elements 'Pitch', 'LaserParam/ProcessingSpeed', 'LaserParam/Power'
        info1 = self.data[idx, 3:4]
        info2 = self.data[idx, 4:5]
        info3 = self.data[idx, 5:6]

        input_data = np.concatenate([mp, info1, info2], axis=0)

        # input_data = self.data[idx, :3]
        output_data = self.data[idx, 49:] #sp: 4:47, color: 47:
        return torch.tensor(input_data, dtype=torch.float32), torch.tensor(output_data, dtype=torch.float32)

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1000)  

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(5, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )
        self._initialize_weights()

    def forward(self, x):
        return self.layers(x)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

In [ ]:
def delta_e_loss(pred, target):
    return torch.sqrt(torch.sum((pred - target) ** 2, dim=-1))

In [ ]:
all_time_best = 1000
all_results = []

for runtime in tqdm(range(5)):
    set_seed(runtime)  

    epochs = 3000

    train_dataset = LabDataset(f'dataset/npy/raw_random_train_sp_color.npy')
    val_dataset = LabDataset(f'dataset/npy/raw_random_val_sp_color.npy')

    train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False)

    model = Model().cuda()

    loss_fn = nn.MSELoss().cuda()
    optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=1e-3)
    lr_schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=5e-4)

    # Initialization
    best_val_loss = float('inf') 

    for epoch in tqdm(range(epochs)): 
        model.train()  
        for inputs, targets in train_dataloader:
            inputs = inputs.cuda()
            targets = targets.cuda()

            # Forward pass
            outputs = model(inputs) 
            # loss = loss_fn(outputs, targets) 
            loss = delta_e_loss(outputs, targets).mean()

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad(): 
            val_losses = []
            for inputs, targets in val_dataloader:
                inputs = inputs.cuda()
                targets = targets.cuda()

                # Forward pass
                outputs = model(inputs)
                # loss = loss_fn(outputs, targets)
                loss = delta_e_loss(outputs, targets).mean()

                val_losses.append(loss.item())

        avg_val_loss = sum(val_losses) / len(val_losses)
        lr_schedule.step()

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), f'best_forward_color_{runtime}.pth')  # save model weights

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Training Loss: {loss.item()}, Validation Loss: {avg_val_loss}, Best Loss: {best_val_loss}')
    
    test_data = np.load(f'dataset/npy/raw_random_test_sp_color.npy')

    mp = test_data[:, :3]  # First three elements 'Pitch', 'LaserParam/ProcessingSpeed', 'LaserParam/Power'
    info1 = test_data[:, 3:4]
    info2 = test_data[:, 4:5]
    info3 = test_data[:, 5:6]

    test_inputs = torch.tensor(np.concatenate([mp, info1, info2], axis=1), dtype=torch.float32)
    test_targets = torch.tensor(test_data[:, 49:], dtype=torch.float32)

    test_dataset = TensorDataset(test_inputs, test_targets)
    test_dataloader = DataLoader(test_dataset, batch_size=1)

    all_delta_e = 0

    model = Model()
    model.load_state_dict(torch.load(f'best_forward_color_{runtime}.pth'))
    model.eval()

    for inputs, targets in tqdm(test_dataloader):
        with torch.no_grad():

            outputs = model(inputs)

            prediction = outputs[0].numpy()
            target = targets[0].numpy()

            delta_e = deltaE_cie76(prediction, target)
            all_delta_e += delta_e

    print(f'Test Delta E {runtime}: {np.round(all_delta_e / len(test_dataloader), 2)}')
    
    all_results.append(np.round(all_delta_e / len(test_dataloader), 2))
    print('-----------------------------------------------')
    
print(f'All: {all_results}')
print(f'Average: {np.round(np.mean(all_results), 2)}')
print(f'STD: {np.round(np.std(all_results), 2)}')
print(f'Min: {np.min(all_results)}')
print(f'Max: {np.max(all_results)}')